# Backpropagation

In [1]:
# This imports numpy, which is a linear algebra library.
import numpy as np

### Variable Definition

* **X**: Input dataset matrix where each row is a training example
* **y**: Output dataset matrix where each row is a training example
* **l0**: First Layer of the Network, specified by the input data
* **l1**: Second Layer of the Network, otherwise known as the hidden layer
* **syn0**: First layer of weights, Synapse 0, connecting l0 to l1.
* *****: Elementwise multiplication, so two vectors of equal size are multiplying corresponding values 1-to-1 to generate a final vector of identical size.
* **-**: Elementwise subtraction, so two vectors of equal size are subtracting corresponding values 1-to-1 to generate a final vector of identical size.
* **x.dot(y)**: If x and y are vectors, this is a dot product. If both are matrices, it's a matrix-matrix multiplication. If only one is a matrix, then it's vector matrix multiplication.

### Sigmoid function

This is our "nonlinearity". While it can be several kinds of functions, this nonlinearity maps a function called a "sigmoid".   

A sigmoid function maps any value to a value between 0 and 1. We use it to convert numbers to probabilities.   
It also has several other desirable properties for training neural networks. 

In [2]:
def nonlin(x, deriv=False):
    if deriv is True :
        return x*(1-x)
    return 1/(1 + np.exp(-x))

![title](../images/simple-sigmoid.png)

Notice that this function can also generate the derivative of a sigmoid (when deriv=True). One of the desirable properties of a sigmoid function is that its output can be used to create its derivative. If the sigmoid's output is a variable "out", then the derivative is simply out * (1-out). This is very efficient. 

If you're unfamililar with derivatives, just think about it as the slope of the sigmoid function at a given point (as you can see above, different points have different slopes). For more on derivatives, check out this [derivatives tutorial from Khan Academy](https://www.khanacademy.org/test-prep/fr-twelveth-grade-math/les-derivees/introduction-aux-derivees/v/calculus-derivatives-1). 

### Input dataset & output dataset  

The first line (X = ...) initializes our input dataset as a numpy matrix. 

* Each row is a single "training example". 
* Each column corresponds to one of our input nodes. 

Thus, we have 3 input nodes to the network and 4 training examples. 

The second line (y = ... ) initializes our output dataset. In this case, I generated the dataset horizontally (with a single row and 4 columns) for space. ".T" is the transpose function. After the transpose, this y matrix has 4 rows with one column. J
ust like our input, each row is a training example, and each column (only one) is an output node.  
So, our network has 3 inputs and 1 output. 

In [3]:
X = np.array([[0, 0, 1],
              [0, 1, 1],
              [1, 0, 1],
              [1, 1, 1]])

y = np.array([[0, 0, 1, 1]]).T

In [4]:
# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)

### Initialize weights randomly with mean 0

This is our weight matrix for this neural network. It's called "syn0" to imply "synapse zero".  
Since we only have 2 layers (input and output), we only need one matrix of weights to connect them.  
Its dimension is (3,1) because we have 3 inputs and 1 output. Another way of looking at it is that `l0` is of size 3 and `l1 ` is of size 1. Thus, we want to connect every node in `l0` to every node in `l1 `, which requires a matrix of dimensionality (3,1).


In [5]:
syn0 = 2*np.random.random((3,1)) - 1

Another note is that the "neural network" is really just this matrix.  
We have "layers" l0 and l1 but they are transient values based on the dataset.  
We don't save them. All of the learning is stored in the syn0 matrix. 

### 2 Layer Neural Network

Here begins our actual network training code.  
This for loop "iterates" multiple times over the training code to optimize our network to the dataset. 

In [6]:
for iter in range(10000):
    # forward propagation
    l0 = X
    l1 = nonlin(np.dot(l0,syn0))

    # how much did we miss?
    l1_error = y - l1

    # multiply how much we missed by the 
    # slope of the sigmoid at the values in l1
    l1_delta = l1_error * nonlin(l1, True)

    # update weights
    syn0 += np.dot(l0.T, l1_delta)

print("Output After Training:")
print(l1)

Output After Training:
[[ 0.00966449]
 [ 0.00786506]
 [ 0.99358898]
 [ 0.99211957]]


## Explanation

```python
l0 = X
```

Since our first layer, `l0`, is simply our data. We explicitly describe it as such at this point.  
Remember that X contains 4 training examples (rows). We're going to process all of them at the same time in this implementation.

This is known as "full batch" training. Thus, we have 4 different `l0` rows, but you can think of it as a single training example if you want. It makes no difference at this point. (We could load in 1000 or 10,000 if we wanted to without changing any of the code). 

```python
l1 = nonlin(np.dot(l0,syn0))
```

This is our prediction step.  
Basically, we first let the network "try" to predict the output given the input.  
We will then study how it performs so that we can adjust it to do a bit better for each iteration. 

This line contains 2 steps. The first matrix multiplies `l0` by `syn0`. The second passes our output through the sigmoid function. Consider the dimensions of each:

*(4 x 3) dot (3 x 1) = (4 x 1)*

Matrix multiplication is ordered, such the dimensions in the middle of the equation must be the same. The final matrix generated is thus the number of rows of the first matrix and the number of columns of the second matrix.

Since we loaded in 4 training examples, we ended up with 4 guesses for the correct answer, a (4 x 1) matrix. Each output corresponds with the network's guess for a given input. Perhaps it becomes intuitive why we could have "loaded in" an arbitrary number of training examples. The matrix multiplication would still work out.

```python
l1_error = y - l1
```

So, given that `l1` had a "guess" for each input. We can now compare how well it did by subtracting the true answer (`y`) from the guess (`l1`). `l1_error` is just a vector of positive and negative numbers reflecting how much the network missed.  


```python
l1_delta = l1_error * nonlin(l1,True)
```

Now we're getting to the good stuff! This is the secret sauce! There's a lot going on in this line, so let's further break it into two parts. 

#### First Part: The Derivative

```python
nonlin(l1,True)
```

If `l1` represents these three dots, the code above generates the slopes of the lines below. Notice that very high values such as *x=2.0 (green dot)* and very low values such as *x=-1.0 (purple dot)* have rather shallow slopes. The highest slope you can have is at *x=0 (blue dot)*. This plays an important role. Also notice that all derivatives are between 0 and 1. 

![title](../images/sigmoid-deriv-2.png)

#### Entire Statement: The Error Weighted Derivative

```python
l1_delta = l1_error * nonlin(l1, True)
```

There are more "mathematically precise" ways than "The Error Weighted Derivative" that captures the intuition. `l1_error` is a (4,1) matrix. `nonlin(l1,True)` returns a (4,1) matrix. What we're doing is multiplying them "elementwise". This returns a (4,1) matrix `l1_delta` with the multiplied values.

When we multiply the "slopes" by the error, we are reducing the error of high confidence predictions. Look at the sigmoid picture again! If the slope was really shallow (close to 0), then the network either had a very high value, or a very low value. This means that the network was quite confident one way or the other. However, if the network guessed something close to (x=0, y=0.5) then it isn't very confident. We update these "wishy-washy" predictions most heavily, and we tend to leave the confident ones alone by multiplying them by a number close to 0. 

```python
syn0 += np.dot(l0.T, l1_delta)
```

We are now ready to update our network! Let's take a look at a single training example. 

![title](../images/toy_network_deriv.png)

In this training example, we're all setup to update our weights. Let's update the far left weight (9.5).

**weight_update = input_value * l1_delta**

For the far left weight, this would multiply 1.0 * the l1_delta. Presumably, this would increment 9.5 ever so slightly. Why only a small ammount? Well, the prediction was already very confident, and the prediction was largely correct. A small error and a small slope means a VERY small update. Consider all the weights. It would ever so slightly increase all three. 

![title](../images/toy_network_batch.png)

However, because we're using a "full batch" configuration, we're doing the above step on all four training examples. So, it looks a lot more like the image above. So, what does `syn0 += np.dot(l0.T,l1_delta)` do? It computes the weight updates for each weight for each training example, sums them, and updates the weights, all in a simple line. Play around with the matrix multiplication and you'll see it do this! 

## Takeaways:

So, now that we've looked at how the network updates, let's look back at our training data and reflect. When both an input and output are 1, we increase the weight between them. When an input is 1 and an output is 0, we decrease the weight between them. 

Thus, in our four training examples below, the weight from the first input to the output would **consistently increment or remain unchanged**, whereas the other two weights would find themselves **both increasing and decreasing across training examples** (cancelling out progress). This phenomenon is what causes our network to learn based on correlations between the input and output. 

Source: [A Neural Network in 11 lines of Python](http://iamtrask.github.io/2015/07/12/basic-python-network/)